In [1]:
#!pip install langchain langchain-openai PyPDF2 langchain-community pypdf chromadb -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.7 MB/s eta 0:

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import os
import glob
import PyPDF2

import os
os.environ['OPENAI_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxxxxxx' # replace with your key

In [3]:
# Initialize embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [4]:
# Define text splitter for chunking
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=100
)

In [5]:
# Note for colab : create a folder inside Sample Data with name pdf
# Directory containing multiple PDF files
pdf_directory = "/content/sample_data/pdf"
#/content/sample_data/pdf
# Load all PDF documents from directory
documents = []
for pdf_path in glob.glob(os.path.join(pdf_directory, "*.pdf")):
    loader = PyPDFLoader(pdf_path)
    pdf_documents = loader.load_and_split(text_splitter=text_splitter)
    documents.extend(pdf_documents)



In [6]:
len(documents)

75

In [7]:
for d in documents:
    print('\nMetaData\n: ',d.metadata)
    print('\nContent\n: ',d.page_content)
    print('\n###########################\n')


MetaData
:  {'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2024-11-09T03:56:33+00:00', 'source': '/content/sample_data/pdf/INTRODUCTION TO EV.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1'}

Content
:  EV OVERVIEW 
An electric vehicle is powered by a battery 
- Edrive and it does not need any type of
fossil fuel. 
While conventional vehicles use
an internal 
combustion engine and gaso -
line or diesel as fuel, electric vehicles use
electrical energy 
that is stored inside the
high voltage 
battery. As a result, electric
vehicles run 
on electricity, they are eco-
friendly - they do not require fuel and are
zero emission vehicles.
Review and Characteristics 
This is an electric vehicle. Some of the ve-
hicle’s systems operate differently and  
have different operating characteristics  
than vehicles equipped with an internal  
combustion engine. It is important to care-
fully read the entire Owner’s Manual for  
this reason. 
This Vehicle uses two ty

In [8]:
# Initialize and store embeddings in ChromaDB
db = Chroma.from_documents(
    documents,
    embedding=embeddings,
    persist_directory="emb6"
)

In [9]:
# Perform similarity search
results = db.similarity_search(
    "What are the main differences between electric and conventional vehicles?",
    k=2
)

In [10]:
results

[Document(metadata={'source': '/content/sample_data/pdf/INTRODUCTION TO EV.pdf', 'page': 0, 'moddate': '2024-11-09T03:56:33+00:00', 'page_label': '1', 'total_pages': 9, 'creationdate': '', 'producer': 'iLovePDF', 'creator': 'PyPDF'}, page_content='EV OVERVIEW \nAn electric vehicle is powered by a battery \n- Edrive and it does not need any type of\nfossil fuel. \nWhile conventional vehicles use\nan internal \ncombustion engine and gaso -\nline or diesel as fuel, electric vehicles use\nelectrical energy \nthat is stored inside the\nhigh voltage \nbattery. As a result, electric\nvehicles run \non electricity, they are eco-\nfriendly - they do not require fuel and are\nzero emission vehicles.\nReview and Characteristics \nThis is an electric vehicle. Some of the ve-\nhicle’s systems operate differently and  \nhave different operating characteristics  \nthan vehicles equipped with an internal  \ncombustion engine. It is important to care-\nfully read the entire Owner’s Manual for  \nthis r

In [11]:
for r in results:
    print('Page No :',r.metadata['page'])
    print('Source :',r.metadata['source'])
    print('page_content :',r.page_content)
    print('*'*40,'\n')

Page No : 0
Source : /content/sample_data/pdf/INTRODUCTION TO EV.pdf
page_content : EV OVERVIEW 
An electric vehicle is powered by a battery 
- Edrive and it does not need any type of
fossil fuel. 
While conventional vehicles use
an internal 
combustion engine and gaso -
line or diesel as fuel, electric vehicles use
electrical energy 
that is stored inside the
high voltage 
battery. As a result, electric
vehicles run 
on electricity, they are eco-
friendly - they do not require fuel and are
zero emission vehicles.
Review and Characteristics 
This is an electric vehicle. Some of the ve-
hicle’s systems operate differently and  
have different operating characteristics  
than vehicles equipped with an internal  
combustion engine. It is important to care-
fully read the entire Owner’s Manual for  
this reason. 
This Vehicle uses two types of battery sys-
tems- a high voltage system in which a  
high voltage battery powers the inverter  
and electric motor which in turn propel the 
vehicl

In [13]:
# Similarity search with score
results_with_scores = db.similarity_search_with_score(
    "What are the main differences between electric and conventional vehicles?" ,
    k=3
)

In [14]:
results_with_scores

[(Document(metadata={'total_pages': 9, 'moddate': '2024-11-09T03:56:33+00:00', 'creationdate': '', 'producer': 'iLovePDF', 'creator': 'PyPDF', 'source': '/content/sample_data/pdf/INTRODUCTION TO EV.pdf', 'page': 0, 'page_label': '1'}, page_content='EV OVERVIEW \nAn electric vehicle is powered by a battery \n- Edrive and it does not need any type of\nfossil fuel. \nWhile conventional vehicles use\nan internal \ncombustion engine and gaso -\nline or diesel as fuel, electric vehicles use\nelectrical energy \nthat is stored inside the\nhigh voltage \nbattery. As a result, electric\nvehicles run \non electricity, they are eco-\nfriendly - they do not require fuel and are\nzero emission vehicles.\nReview and Characteristics \nThis is an electric vehicle. Some of the ve-\nhicle’s systems operate differently and  \nhave different operating characteristics  \nthan vehicles equipped with an internal  \ncombustion engine. It is important to care-\nfully read the entire Owner’s Manual for  \nthis 

In [15]:
for document, score in results_with_scores:
    page = document.metadata.get("page", "N/A")
    source = document.metadata.get("source", "N/A")
    page_content = document.page_content

    print(f"Page: {page}")
    print(f"Source: {source}")
    print(f"Content: {page_content}")
    print(f"Similarity Score: {score}")
    print("\n" + "-" * 40 + "\n")

Page: 0
Source: /content/sample_data/pdf/INTRODUCTION TO EV.pdf
Content: EV OVERVIEW 
An electric vehicle is powered by a battery 
- Edrive and it does not need any type of
fossil fuel. 
While conventional vehicles use
an internal 
combustion engine and gaso -
line or diesel as fuel, electric vehicles use
electrical energy 
that is stored inside the
high voltage 
battery. As a result, electric
vehicles run 
on electricity, they are eco-
friendly - they do not require fuel and are
zero emission vehicles.
Review and Characteristics 
This is an electric vehicle. Some of the ve-
hicle’s systems operate differently and  
have different operating characteristics  
than vehicles equipped with an internal  
combustion engine. It is important to care-
fully read the entire Owner’s Manual for  
this reason. 
This Vehicle uses two types of battery sys-
tems- a high voltage system in which a  
high voltage battery powers the inverter  
and electric motor which in turn propel the 
vehicle and a low